## Домашнее задание по теме: "Улучшение качества модели"

Описание задания:

В домашнем задании нужно решить задачу классификации наличия болезни сердца у пациентов наиболее эффективно. Данные для обучения моделей необходимо загрузить самостоятельно с сайта. Целевая переменная – наличие болезни сердца (HeartDisease). Она принимает значения 0 или 1 в зависимости от отсутствия или наличия болезни соответственно. Подробное описание признаков можно прочесть в описании датасета на сайте. Для выполнения работы не обязательно вникать в медицинские показатели.

### 1. Получите данные и загрузите их в рабочую среду

In [ ]:
# загрузка csv в google colab
from google.colab import files
uploaded = files.upload()

Saving heart.csv to heart.csv


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('./heart.csv')
df.head(5)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [ ]:
print(df.isna().sum())

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64


### 2. Подготовьте датасет к обучению моделей

a) Категориальные переменные переведите в цифровые значения. Можно использовать pd.get_dummies, preprocessing.LabelEncoder. Старайтесь не использовать для этой задачи циклы.

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

df['Sex'] = le.fit_transform(df['Sex'])
df['ExerciseAngina'] = le.fit_transform(df['ExerciseAngina'])

df = pd.get_dummies(df, columns=['ChestPainType', 'RestingECG', 'ST_Slope'])

df.head(5)

,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,HeartDisease,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,40,1,140,289,0,172,0,0.0,0,0,1,0,0,0,1,0,0,0,1
1,49,0,160,180,0,156,0,1.0,1,0,0,1,0,0,1,0,0,1,0
2,37,1,130,283,0,98,0,0.0,0,0,1,0,0,0,0,1,0,0,1
3,48,0,138,214,0,108,1,1.5,1,1,0,0,0,0,1,0,0,1,0
4,54,1,150,195,0,122,0,0.0,0,0,0,1,0,0,1,0,0,0,1


### 3. Разделите выборку на обучающее и тестовое подмножество. 80% данных оставить на обучающее множество, 20% на тестовое.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop(['HeartDisease'], axis=1), df['HeartDisease'], random_state=42, test_size=0.2)

### 4. Обучите модель логистической регрессии с параметрами по умолчанию

In [ ]:
# отклчюаю warning log, так как при дефолтном значении max_iter сыпет ошибками
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)

LogisticRegression(random_state=42)

### 5. Подсчитайте основные метрики модели. Используйте следующие метрики и функцию:
```python
cross_validate(…, cv=10, scoring=['accuracy','recall','precision','f1'])
```

In [ ]:
from sklearn.model_selection import cross_validate
lr_cv = cross_validate(lr, X_train, y_train, cv=10, scoring=['accuracy','recall','precision','f1'])
lr_cv

{'fit_time': array([0.02219296, 0.02403402, 0.02189112, 0.02155781, 0.02262187,
        0.02169514, 0.02349496, 0.02282405, 0.02220511, 0.04636645]),
 'score_time': array([0.00768518, 0.00708175, 0.00726104, 0.00727272, 0.00730133,
        0.0087676 , 0.00769424, 0.00715303, 0.00698256, 0.01354837]),
 'test_accuracy': array([0.93243243, 0.87837838, 0.89189189, 0.85135135, 0.8630137 ,
        0.89041096, 0.76712329, 0.87671233, 0.89041096, 0.82191781]),
 'test_recall': array([0.925     , 0.875     , 0.875     , 0.90243902, 0.9       ,
        0.925     , 0.825     , 0.9       , 0.95      , 0.875     ]),
 'test_precision': array([0.94871795, 0.8974359 , 0.92105263, 0.84090909, 0.85714286,
        0.88095238, 0.76744186, 0.87804878, 0.86363636, 0.81395349]),
 'test_f1': array([0.93670886, 0.88607595, 0.8974359 , 0.87058824, 0.87804878,
        0.90243902, 0.79518072, 0.88888889, 0.9047619 , 0.84337349])}

### 6. Оптимизируйте 3-4 параметра модели:

a) Используйте GridSearchCV.

b) Используйте RandomizedSearchCV.

c) *Добавьте в п. 6b 2-5 моделей классификации и вариации их параметров.

d) Повторите п. 5 после каждого итогового изменения параметров.

In [ ]:
# отключаем warning'и, т.к. ругается при дефолтных параметрах lr на max_iter
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression (random_state=42)

lr_params = [{'C':        [0.1, 1, 10],
              'penalty' : ['l1', 'l2'],
              'max_iter': range(1000, 10000, 1000),
              'solver':   ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky']}]

In [ ]:
# LogisticRegression + GridSearchCV

from sklearn.model_selection import GridSearchCV
gs_lr = GridSearchCV(lr, param_grid=lr_params)

In [ ]:
gs_lr_cv = cross_validate(gs_lr, X_train, y_train, cv=10, scoring=['accuracy','recall','precision','f1'])
gs_lr_cv

{'fit_time': array([30.00037289, 24.41339588, 27.75922418, 29.03856874, 28.60356355,
        27.43064213, 21.79810882, 23.83781385, 24.38438821, 27.84267163]),
 'score_time': array([0.00682569, 0.00682664, 0.0068512 , 0.0135107 , 0.00664449,
        0.00649118, 0.00675654, 0.00633526, 0.00651884, 0.00658083]),
 'test_accuracy': array([0.91891892, 0.87837838, 0.89189189, 0.86486486, 0.83561644,
        0.89041096, 0.76712329, 0.84931507, 0.89041096, 0.83561644]),
 'test_recall': array([0.925     , 0.875     , 0.875     , 0.90243902, 0.875     ,
        0.925     , 0.825     , 0.85      , 0.95      , 0.875     ]),
 'test_precision': array([0.925     , 0.8974359 , 0.92105263, 0.86046512, 0.83333333,
        0.88095238, 0.76744186, 0.87179487, 0.86363636, 0.83333333]),
 'test_f1': array([0.925     , 0.88607595, 0.8974359 , 0.88095238, 0.85365854,
        0.90243902, 0.79518072, 0.86075949, 0.9047619 , 0.85365854])}

In [ ]:
# LogisticRegression + RandomizedSearchCV

from sklearn.model_selection import RandomizedSearchCV
rs_lr = RandomizedSearchCV(lr, param_distributions=lr_params)

In [ ]:
rs_lr_cv = cross_validate(rs_lr, X_train, y_train, cv=10, scoring=['accuracy','recall','precision','f1'])
rs_lr_cv

{'fit_time': array([1.73522925, 1.22609568, 2.30160761, 0.85104918, 1.05738163,
        1.63817883, 0.64921641, 1.33345795, 1.28433967, 1.78316092]),
 'score_time': array([0.0062511 , 0.00652814, 0.00855565, 0.00836134, 0.01104736,
        0.0072751 , 0.00704288, 0.00802517, 0.00786114, 0.00645947]),
 'test_accuracy': array([0.91891892, 0.87837838, 0.89189189, 0.86486486, 0.82191781,
        0.89041096, 0.76712329, 0.87671233, 0.89041096, 0.83561644]),
 'test_recall': array([0.925     , 0.875     , 0.875     , 0.90243902, 0.875     ,
        0.925     , 0.825     , 0.9       , 0.95      , 0.875     ]),
 'test_precision': array([0.925     , 0.8974359 , 0.92105263, 0.86046512, 0.81395349,
        0.88095238, 0.76744186, 0.87804878, 0.86363636, 0.83333333]),
 'test_f1': array([0.925     , 0.88607595, 0.8974359 , 0.88095238, 0.84337349,
        0.90243902, 0.79518072, 0.88888889, 0.9047619 , 0.85365854])}

In [ ]:
# RandomForest + RandomizedSearchCV

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

from scipy.stats import randint
rf_params = {'n_estimators': randint(10, 100),
             'max_depth': [3, 5, 7, 9],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4]}

rs_rf = RandomizedSearchCV(rf, param_distributions=rf_params)

rs_rf_cv = cross_validate(rs_rf, X_train, y_train, cv=10, scoring=['accuracy','recall','precision','f1'])
rs_rf_cv

{'fit_time': array([5.7097888 , 3.6029644 , 5.39077997, 4.94091773, 4.92540312,
        4.34039474, 4.31405163, 3.62552118, 4.75003839, 4.16539216]),
 'score_time': array([0.01327991, 0.00936151, 0.00952911, 0.01362109, 0.01570296,
        0.00619531, 0.01028419, 0.01688123, 0.01294851, 0.01182914]),
 'test_accuracy': array([0.87837838, 0.89189189, 0.90540541, 0.89189189, 0.82191781,
        0.87671233, 0.80821918, 0.8630137 , 0.87671233, 0.83561644]),
 'test_recall': array([0.925     , 0.875     , 0.9       , 0.92682927, 0.9       ,
        0.925     , 0.9       , 0.875     , 0.95      , 0.925     ]),
 'test_precision': array([0.86046512, 0.92105263, 0.92307692, 0.88372093, 0.8       ,
        0.86046512, 0.7826087 , 0.875     , 0.84444444, 0.80434783]),
 'test_f1': array([0.89156627, 0.8974359 , 0.91139241, 0.9047619 , 0.84705882,
        0.89156627, 0.8372093 , 0.875     , 0.89411765, 0.86046512])}

In [ ]:
# GradientBoosting + RandomizedSearchCV

from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import uniform

gb = GradientBoostingClassifier()

gb_params = {'n_estimators': [50, 100, 150, 200],
             'max_depth': [2, 3, 4, 5],
             'learning_rate': uniform(0.01, 0.1),
             'subsample': uniform(0.5, 0.5)}

gb_rf = RandomizedSearchCV(gb, param_distributions=gb_params)

gb_rf_cv = cross_validate(gb_rf, X_train, y_train, cv=10, scoring=['accuracy','recall','precision','f1'])
gb_rf_cv

{'fit_time': array([ 7.40142822,  9.46522141, 10.61053753,  7.63963604, 11.06785226,
         8.10734749, 11.05766082,  9.7626946 , 10.80388069,  9.50814486]),
 'score_time': array([0.00611377, 0.00566554, 0.00684285, 0.00593042, 0.00599432,
        0.00562882, 0.00978518, 0.00597334, 0.0058682 , 0.00630426]),
 'test_accuracy': array([0.89189189, 0.89189189, 0.89189189, 0.91891892, 0.82191781,
        0.87671233, 0.80821918, 0.83561644, 0.90410959, 0.83561644]),
 'test_recall': array([0.9       , 0.875     , 0.875     , 0.97560976, 0.9       ,
        0.9       , 0.875     , 0.85      , 0.975     , 0.9       ]),
 'test_precision': array([0.9       , 0.92105263, 0.92105263, 0.88888889, 0.8       ,
        0.87804878, 0.79545455, 0.85      , 0.86666667, 0.81818182]),
 'test_f1': array([0.9       , 0.8974359 , 0.8974359 , 0.93023256, 0.84705882,
        0.88888889, 0.83333333, 0.85      , 0.91764706, 0.85714286])}

### 7. Сформулируйте выводы по проделанной работе:
a) Сравните метрики построенных моделей.

b) *Сравните с полученными результатами в домашнем задании по теме «Ансамблирование».

In [ ]:
import numpy as np

print(f'''LogisticRegression: {np.mean(lr_cv['test_accuracy'])}
LogisticRegression + GridSearch: {np.mean(gs_lr_cv['test_accuracy'])}
LogisticRegression + RandomSearch: {np.mean(rs_lr_cv['test_accuracy'])}
RandomForest + RandomSearch: {np.mean(rs_rf_cv['test_accuracy'])}
GradientBoosting + RandomSearch: {np.mean(gb_rf_cv['test_accuracy'])}''')

LogisticRegression: 0.8663643095149943
LogisticRegression + GridSearch: 0.8622547204738986
LogisticRegression + RandomSearch: 0.8636245834875972
RandomForest + RandomSearch: 0.8649759348389485
GradientBoosting + RandomSearch: 0.8676786375416512
